<a href="https://colab.research.google.com/github/preetamjumech/pytorch/blob/main/pytorch_training_pipeline_using_nn_module_09_12_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [5]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [7]:
X_train_tensor = torch.from_numpy(X_train.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train.astype(np.float32))
y_test_tensor = torch.from_numpy(y_test.astype(np.float32))

In [8]:
import torch.nn as nn

class MySimpleNN(nn.Module):
  def __init__(self,num_features):
    super().__init__()
    self.linear = nn.Linear(num_features,1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):
    out = self.linear(features)
    out = self.sigmoid(out)
    return out

In [9]:
learning_rate = 0.1
epochs = 25

In [10]:
loss_function = nn.BCELoss()

In [14]:
model = MySimpleNN(X_train_tensor.shape[1])
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

for epoch in range(epochs):
  y_pred = model(X_train_tensor)
  loss = loss_function(y_pred, y_train_tensor.view(-1,1))
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 0.5732699632644653
Epoch: 2, Loss: 0.45606905221939087
Epoch: 3, Loss: 0.3906588852405548
Epoch: 4, Loss: 0.34851428866386414
Epoch: 5, Loss: 0.3186386227607727
Epoch: 6, Loss: 0.2960812747478485
Epoch: 7, Loss: 0.2782802879810333
Epoch: 8, Loss: 0.26376789808273315
Epoch: 9, Loss: 0.25163784623146057
Epoch: 10, Loss: 0.24129778146743774
Epoch: 11, Loss: 0.2323426455259323
Epoch: 12, Loss: 0.2244851291179657
Epoch: 13, Loss: 0.21751531958580017
Epoch: 14, Loss: 0.2112758457660675
Epoch: 15, Loss: 0.20564612746238708
Epoch: 16, Loss: 0.20053204894065857
Epoch: 17, Loss: 0.19585898518562317
Epoch: 18, Loss: 0.19156678020954132
Epoch: 19, Loss: 0.18760649859905243
Epoch: 20, Loss: 0.1839376538991928
Epoch: 21, Loss: 0.18052656948566437
Epoch: 22, Loss: 0.17734487354755402
Epoch: 23, Loss: 0.17436851561069489
Epoch: 24, Loss: 0.1715768277645111
Epoch: 25, Loss: 0.1689520925283432


In [15]:
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.5).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.5369344353675842
